In [4]:
%load_ext autoreload
%autoreload 2

from ProblemEvaluator import ProblemEvaluator
import numpy as np
from pymoo.problems import get_problem
import matplotlib.pyplot as plt
from optimisation.model.population import Population
from features.RandomWalkAnalysis import RandomWalkAnalysis
from features.GlobalAnalysis import GlobalAnalysis
import time

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Flags for which feature set we want to test.
sample_rw = True
sample_aw = False

# Example problem.
num_samples = 1
num_cores = 1
n_var = 3
problem_name = "MW11"
problem = get_problem(problem_name, n_var)
instance_string = f"{problem_name}_d{n_var}"

# Generate evaluator which can generate RW samples and LHS samples.
evaluator = ProblemEvaluator(
    problem, instance_string, "eval", "test_results", num_cores, num_samples
)
evaluator.initialize_number_of_cores()
pre_sampler = evaluator.create_pre_sampler()
pre_sampler.create_pregen_sample_dir()
pre_sampler.create_pops_dir(instance_string, "test_pops")

Initialising evaluator in eval mode.
Starting the creation process for population directories for problem: MW11_d3 in mode: eval
Directory already exists and does not need to be created: test_pops/MW11_d3/eval/global
Directory already exists and does not need to be created: test_pops/MW11_d3/eval/rw
Population directories setup complete.
Global population directory: test_pops/MW11_d3/eval/global
RW population directory: test_pops/MW11_d3/eval/rw


### Random Walk


In [43]:
sample_number = 1
walk_number = 1

import time  # Import the time module

# Start timing for reading walk neighbours
start_time = time.time()
walk, neighbours = pre_sampler.read_walk_neighbours(sample_number, walk_number)
end_time = time.time()
print(f"Reading walk neighbours took {end_time - start_time:.2f} seconds.")

# Start timing for generating walk neighbour populations

Reading walk neighbours took 0.00 seconds.


In [5]:
start_time = time.time()
pop_walk, pop_neighbours_list = evaluator.generate_walk_neig_populations(
    problem, walk, neighbours, True
)
end_time = time.time()
print(
    f"Generating walk neighbour populations took {end_time - start_time:.2f} seconds."
)

Generating walk neighbour populations took 4.56 seconds.


In [44]:
# Start timing for saving walk neighbour population
start_time = time.time()
pre_sampler.save_walk_neig_population(
    pop_walk, pop_neighbours_list, sample_number, walk_number
)
end_time = time.time()
print(
    f"Saving walk neighbour population (new method) took {end_time - start_time:.2f} seconds."
)

NameError: name 'pop_walk' is not defined

In [45]:
# Attempt to use pre-generated samples.
start_time = time.time()
pop_walk, pop_neighbours_list = pre_sampler.load_walk_neig_population(
    sample_number, walk_number
)
end_time = time.time()
print(
    f"Loading walk neighbour population (new method) took {end_time - start_time:.2f} seconds."
)

[2024-02-22 08:43:53] Loaded walk and individual neighbours for sample 1, walk 1 from test_pops/MW11_d3/eval/rw/sample1.
Loading walk neighbour population (new method) took 0.32 seconds.


#### Analysis Object Creation


In [46]:
rw_norm_values = evaluator.compute_rw_normalisation_values(pre_sampler, problem)
rw_analysis = RandomWalkAnalysis(
    pop_walk, pop_neighbours_list, rw_norm_values, evaluator.results_dir
)

[2024-02-22 08:43:55] Initialising normalisation computations for RW samples using 1 cores. This requires full evaluation of the entire sample set and may take some time while still being memory-efficient.
[2024-02-22 08:43:56] Loaded walk and individual neighbours for sample 1, walk 1 from test_pops/MW11_d3/eval/rw/sample1.
[2024-02-22 08:43:56] Loaded walk and individual neighbours for sample 1, walk 2 from test_pops/MW11_d3/eval/rw/sample1.
[2024-02-22 08:43:57] Loaded walk and individual neighbours for sample 1, walk 3 from test_pops/MW11_d3/eval/rw/sample1.
[2024-02-22 08:43:57] Evaluated the normalisation values for this sample set in 1.95 seconds.



#### Running each step of RW feature eval


In [47]:
start_time = time.time()
rw_analysis.preprocess_nans_on_walks()
end_time = time.time()
print(f"Preprocessing nans on walks took {end_time - start_time:.2f} seconds.")

Preprocessing nans on walks took 0.28 seconds.


In [48]:
# Timing each of the features eval
start_time = time.time()
_ = rw_analysis.compute_solver_related_features()
end_time = time.time()
print(f"RW solver features took {end_time - start_time:.2f} seconds.")

RW solver features took 0.17 seconds.


In [49]:
start_time = time.time()
_ = rw_analysis.compute_neighbourhood_distance_features(norm_method="95th")
end_time = time.time()
print(f"RW neighbourhood dist features took {end_time - start_time:.2f} seconds.")

RW neighbourhood dist features took 0.20 seconds.


In [50]:
start_time = time.time()
_ = rw_analysis.compute_uncons_neighbourhood_hv_features(norm_method="95th")
end_time = time.time()
print(f"RW uncons neighbourhood HV features took {end_time - start_time:.2f} seconds.")

RW uncons neighbourhood HV features took 0.22 seconds.


In [51]:
start_time = time.time()
_ = rw_analysis.compute_cons_neighbourhood_hv_features(norm_method="95th")
end_time = time.time()
print(f"RW cons neighbourhood HV features took {end_time - start_time:.2f} seconds.")

RW cons neighbourhood HV features took 0.05 seconds.


In [52]:
start_time = time.time()
_ = rw_analysis.compute_neighbourhood_violation_features(norm_method="95th")
end_time = time.time()
print(f"RW neighbourhood violation features took {end_time - start_time:.2f} seconds.")

RW neighbourhood violation features took 0.20 seconds.


In [53]:
start_time = time.time()
_ = rw_analysis.compute_neighbourhood_dominance_features()
end_time = time.time()
print(f"RW neighbourhood dominance features took {end_time - start_time:.2f} seconds.")

RW neighbourhood dominance features took 0.41 seconds.


In [54]:
start_time = time.time()
_ = GlobalAnalysis.compute_ic_features(rw_analysis.pop_walk, sample_type="rw")
end_time = time.time()
print(f"RW IC features took {end_time - start_time:.2f} seconds.")

RW IC features took 0.27 seconds.


### Global


In [5]:
sample_number = 1
# Start timing for reading walk neighbours
start_time = time.time()
global_sample = pre_sampler.read_global_sample(sample_number)
end_time = time.time()
print(f"Reading global sample took {end_time - start_time:.2f} seconds.")

Global sample size for 3d, no. 1: 3000
Reading global sample took 0.00 seconds.


In [6]:
start_time = time.time()
pop_global = evaluator.generate_global_population(problem, global_sample, True)
end_time = time.time()
print(f"Generating global populations took {end_time - start_time:.2f} seconds.")

Generating global populations took 25.53 seconds.


In [7]:
# Start timing for saving walk neighbour population
start_time = time.time()
pre_sampler.save_global_population(pop_global, sample_number)
end_time = time.time()
print(
    f"Saving global population (new method) took {end_time - start_time:.2f} seconds."
)

Saving global population (new method) took 0.12 seconds.


In [8]:
# Attempt to use pre-generated samples.
start_time = time.time()
pop_global = pre_sampler.load_global_population(sample_number)
end_time = time.time()
print(
    f"Loading global population (new method) took {end_time - start_time:.2f} seconds."
)

[2024-02-22 08:37:03] Global population for sample 1 loaded from test_pops/MW11_d3/eval/global/pop_global_1.pkl. Length: 3000
Loading global population (new method) took 0.04 seconds.


#### Analysis Object Creation


In [9]:
global_norm_values = evaluator.compute_global_normalisation_values(pre_sampler, problem)
global_analysis = GlobalAnalysis(pop_global, global_norm_values, evaluator.results_dir)

[2024-02-22 08:37:03] Initialising normalisation computations for global samples with 1 processes. This requires full evaluation of the entire sample set and may take some time while still being memory-efficient.


[2024-02-22 08:37:04] Global population for sample 1 loaded from test_pops/MW11_d3/eval/global/pop_global_1.pkl. Length: 3000
[2024-02-22 08:37:04] Evaluated the normalisation values for this sample set in 1.19 seconds.



In [27]:
print(global_analysis.pop.extract_var().shape)
print(global_analysis.pop.extract_obj().shape)
print(global_analysis.pop.extract_rank().shape)
print(global_analysis.pop.extract_uncons_rank().shape)
print(global_analysis.pop.extract_cv().shape)
print(global_analysis.pop.extract_cons().shape)

(3000, 3)
(3000, 2)
(3000,)
(3000,)
(3000, 1)
(3000, 4)


#### Running each step of Global feature eval


In [ ]:
from features.Analysis import Analysis

global_analysis.pop = global_analysis.pop.remove_nan_inf_rows(
    "global", re_evaluate=True
)

In [28]:
start_time = time.time()
_ = global_analysis.compute_fsr()
end_time = time.time()
print(f"Global feasibility features took {end_time - start_time:.2f} seconds.")

Global feasibility features took 0.00 seconds.


In [29]:
start_time = time.time()
_ = global_analysis.corr_obj()
end_time = time.time()
print(f"Global obj corr features took {end_time - start_time:.2f} seconds.")

Global obj corr features took 0.01 seconds.


In [30]:
start_time = time.time()
_ = global_analysis.obj_kurt()
_ = global_analysis.obj_skew()
end_time = time.time()
print(f"Global obj skew, kurtosis features took {end_time - start_time:.2f} seconds.")

Global obj skew, kurtosis features took 0.02 seconds.


In [31]:
start_time = time.time()
_ = global_analysis.uc_rk_distr()
end_time = time.time()
print(f"Global uc rk distr features took {end_time - start_time:.2f} seconds.")

Global uc rk distr features took 0.00 seconds.


In [32]:
start_time = time.time()
_ = global_analysis.cv_distr(norm_method="95th")
end_time = time.time()
print(f"Global cv distr features took {end_time - start_time:.2f} seconds.")

Global cv distr features took 0.00 seconds.


In [33]:
start_time = time.time()
_ = global_analysis.PiIZ()
end_time = time.time()
print(f"Global ideal zone features took {end_time - start_time:.2f} seconds.")

Global ideal zone features took 0.02 seconds.


In [34]:
start_time = time.time()
_ = global_analysis.compute_ps_pf_distances(norm_method="95th")
end_time = time.time()
print(f"Global PS PF distance features took {end_time - start_time:.2f} seconds.")

Global PS PF distance features took 0.06 seconds.


In [35]:
start_time = time.time()
_ = global_analysis.compute_PF_UPF_features(norm_method="95th")
end_time = time.time()
print(f"Global PF UPF features took {end_time - start_time:.2f} seconds.")

Global PF UPF features took 0.04 seconds.


In [36]:
start_time = time.time()
_ = global_analysis.dist_corr()
end_time = time.time()
print(f"Global dist corr features took {end_time - start_time:.2f} seconds.")

Global dist corr features took 0.01 seconds.


In [37]:
start_time = time.time()
_ = global_analysis.compute_ranks_cv_corr()
end_time = time.time()
print(f"Global ranks cv corr features took {end_time - start_time:.2f} seconds.")

Global ranks cv corr features took 0.01 seconds.


In [38]:
start_time = time.time()
_ = global_analysis.rk_uc_var_mdl()
end_time = time.time()
print(f"Global rk uc var mdl features took {end_time - start_time:.2f} seconds.")

Global rk uc var mdl features took 0.01 seconds.


In [39]:
start_time = time.time()
_ = global_analysis.cv_var_mdl()
end_time = time.time()
print(f"Global cv var mdl features took {end_time - start_time:.2f} seconds.")

Global cv var mdl features took 0.00 seconds.


In [40]:
start_time = time.time()
_ = GlobalAnalysis.compute_ic_features(global_analysis.pop, sample_type="global")
end_time = time.time()
print(f"Global IC features took {end_time - start_time:.2f} seconds.")

Global IC features took 1.05 seconds.


In [10]:
import time
import pickle
import numpy as np
import os

# Assuming pre_sampler, evaluator, and problem are already defined
# and generate_global_population method returns pop_global as a numpy array or a list that can be converted to a numpy array

# Generate distributed samples and evaluate populations on these samples.
sample = pre_sampler.read_global_sample(5)
start_time = time.time()
pop_global = evaluator.generate_global_population(problem, sample, eval_fronts=False)
end_time = time.time()

evaluation_time = end_time - start_time  # Calculate total evaluation time in seconds
print(
    f"Total evaluation time: {evaluation_time:.2f} seconds, Sample size: {len(pop_global)}"
)

# Save with pickle
pickle_start_time = time.time()
with open("pop_global.pkl", "wb") as file:
    pickle.dump(pop_global, file)
pickle_end_time = time.time()
pickle_time = pickle_end_time - pickle_start_time

# Save with numpy
numpy_start_time = time.time()
np.save("pop_global.npy", pop_global)
numpy_end_time = time.time()
numpy_time = numpy_end_time - numpy_start_time

print(f"Time to save with pickle: {pickle_time:.2f} seconds")
print(f"Time to save with numpy: {numpy_time:.2f} seconds")

Total evaluation time: 47.82 seconds, Sample size: 10000
Time to save with pickle: 0.12 seconds
Time to save with numpy: 0.14 seconds


In [12]:
import time
import pickle
import numpy as np

# Read with pickle
pickle_read_start_time = time.time()
with open("pop_global.pkl", "rb") as file:
    pop_global_pickle = pickle.load(file)
pickle_read_end_time = time.time()
pickle_read_time = pickle_read_end_time - pickle_read_start_time

print(f"Time to read with pickle: {pickle_read_time:.2f} seconds")

Time to read with pickle: 0.13 seconds


In [ ]:
# IC features
start_ic_time = time.time()

# Compute IC features
(H_max, eps_s, m0, eps05) = compute_ic_features(pop_global)

# End time for computing IC features
end_ic_time = time.time()

# Calculate the computation time
ic_computation_time = end_ic_time - start_ic_time

# Get the size of pop_global
sample_size = len(pop_global)

# Print the descriptive message
print(
    f"Computed IC features in {ic_computation_time:.2f} seconds. Sample size: {sample_size}."
)

Computed IC features in 3.93 seconds. Sample size: 10000.
